In [ ]:
import argparse
import requests
import cv2
import os
from requests import exceptions

ap = argparse.ArgumentParser()
ap.add_argument("-q", "--query", required=True)
ap.add_argument("-o", "--output", required=True)
args = vars(ap.parse_args())

API_KEY = "YOUR_BING_API_KEY"
URL = "https://api.cognitive.microsoft.com/bing/v7.0/images/search"
MAX_RESULTS = 500
GROUP_SIZE = 50

EXCEPTIONS = (
    IOError, FileNotFoundError,
    exceptions.RequestException,
    exceptions.HTTPError,
    exceptions.ConnectionError,
    exceptions.Timeout
)

os.makedirs(args["output"], exist_ok=True)

headers = {
    "Ocp-Apim-Subscription-Key": API_KEY,
    "User-Agent": "Mozilla/5.0"
}
params = {"q": args["query"], "offset": 0, "count": GROUP_SIZE}
print("[INFO] searching Bing API...")
search = requests.get(URL, headers=headers, params=params)
search.raise_for_status()
results = search.json()

estNumResults = min(results["totalEstimatedMatches"], MAX_RESULTS)
total = 0
for offset in range(0, estNumResults, GROUP_SIZE):
    params["offset"] = offset
    results = requests.get(URL, headers=headers, params=params).json()
    for v in results["value"]:
        try:
            r = requests.get(v["contentUrl"], timeout=10)
            ext = os.path.splitext(v["contentUrl"].split("?")[0])[1]
            if ext.lower() not in [".jpg", ".png", ".jpeg"]:
                continue
            p = os.path.join(args["output"], f"{str(total).zfill(8)}{ext}")
            with open(p, "wb") as f:
                f.write(r.content)
            image = cv2.imread(p)
            if image is None:
                os.remove(p)
                continue
            total += 1
        except EXCEPTIONS:
            continue

usage: ipykernel_launcher.py [-h] -q QUERY -o OUTPUT
ipykernel_launcher.py: error: the following arguments are required: -q/--query, -o/--output


SystemExit: 2

train_mask_detector.py

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os
ap = argparse.ArgumentParser()
ap.add_argument("-d", "--dataset", required=True)
ap.add_argument("-m", "--model", default="mask_detector.model")
ap.add_argument("-p", "--plot", default="plot.png")
args = vars(ap.parse_args())
INIT_LR = 1e-4
EPOCHS = 20
BS = 32
imagePaths = list(paths.list_images(args["dataset"]))
data, labels = [], []
for path in imagePaths:
    label = path.split(os.path.sep)[-2]
    image = load_img(path, target_size=(224, 224))
    image = preprocess_input(img_to_array(image))
    data.append(image)
    labels.append(label)
data = np.array(data, dtype="float32")
labels = LabelBinarizer().fit_transform(labels)
labels = to_categorical(labels)
(trainX, testX, trainY, testY) = train_test_split(
    data, labels, test_size=0.2, stratify=labels, random_state=42
)
aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)
baseModel = MobileNetV2(weights="imagenet", include_top=False,
                        input_tensor=Input(shape=(224, 224, 3)))
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten()(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

model = Model(inputs=baseModel.input, outputs=headModel)

for layer in baseModel.layers:
    layer.trainable = False

model.compile(
    loss="categorical_crossentropy",
    optimizer=Adam(learning_rate=INIT_LR),
    metrics=["accuracy"]
)

H = model.fit(
    aug.flow(trainX, trainY, batch_size=BS),
    validation_data=(testX, testY),
    epochs=EPOCHS
)

predIdxs = np.argmax(model.predict(testX), axis=1)
print(classification_report(testY.argmax(axis=1), predIdxs))

model.save(args["model"])

plt.plot(H.history["loss"], label="train_loss")
plt.plot(H.history["val_loss"], label="val_loss")
plt.plot(H.history["accuracy"], label="train_acc")
plt.plot(H.history["val_accuracy"], label="val_acc")
plt.legend()
plt.savefig(args["plot"])

ImportError: Traceback (most recent call last):
  File "c:\Users\BALAJI MURUGAN\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: A dynamic link library (DLL) initialization routine failed.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

detect_mask_image.py

In [20]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import argparse
import cv2
import os

ap = argparse.ArgumentParser()
ap.add_argument("-i", "--image", required=True)
ap.add_argument("-f", "--face", default="face_detector")
ap.add_argument("-m", "--model", default="mask_detector.model")
ap.add_argument("-c", "--confidence", type=float, default=0.5)
args = vars(ap.parse_args())

print("[INFO] loading models...")
net = cv2.dnn.readNet(
    os.path.join(args["face"], "deploy.prototxt"),
    os.path.join(args["face"], "res10_300x300_ssd_iter_140000.caffemodel")
)
model = load_model(args["model"])

image = cv2.imread(args["image"])
(h, w) = image.shape[:2]

blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300),
                             (104.0, 177.0, 123.0))
net.setInput(blob)
detections = net.forward()

for i in range(detections.shape[2]):
    confidence = detections[0, 0, i, 2]
    if confidence > args["confidence"]:
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")

        face = image[startY:endY, startX:endX]
        face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        face = cv2.resize(face, (224, 224))
        face = preprocess_input(img_to_array(face))
        face = np.expand_dims(face, axis=0)

        (mask, withoutMask) = model.predict(face)[0]
        label = "Mask" if mask > withoutMask else "No Mask"
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

        cv2.putText(image, label, (startX, startY - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(image, (startX, startY), (endX, endY), color, 2)

cv2.imshow("Output", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

ImportError: DLL load failed while importing _multiarray_umath: The specified module could not be found.

ImportError: DLL load failed while importing _multiarray_umath: The specified module could not be found.

ImportError: numpy.core._multiarray_umath failed to import

ImportError: numpy.core.umath failed to import

TypeError: Unable to convert function return value to a Python type! The signature was
	() -> handle